In [8]:
import gym
import math
import random
import numpy as np
import matplotlib
import pandas as pd

env = gym.make('Marvin-v0')

 I want to create the neuronNetwork that will take an observation of a enviroment (24 dimensional vector) and returns
 me an action (4 dimensional vector) I then send this action to env.step and receive a reward. Somehow i need to update
 the weights of my network using the reward. My reward should be as big as possible. (1 - reward) is my error? 
 Looks like so. 
 I need three layers of neurons. each neuron has a weight property, activation function, function to update the weight
 Each layer consist of several neurons. I need to use the weight inside the neuron. so layers just run neurons commands
 and play as a coordinator.




In [9]:
def mult_and_sum(a, b):
    result = 0
    for i in range(len(a)):
        result += a[i] * b[i]
    return result


class Neuron(object):
    def __init__(self, input_counts = 1, activation_function = math.tanh, weights = None,
                update_function=None, learning_rate = 0.001, input_layer = False, value = None):
        self.input_layer = input_layer
        if self.input_layer == False:
            self.weights = [random.uniform(-1, 1)] * input_counts if weights == None else weights
        self.value = value
        self.activation_function = activation_function
        self.update_function = self.update_weights if update_function == None else update_function
        self.learning_rate = learning_rate
    
    def setValue(self, value):
        self.value = value
        

    def __str__(self):
        message = "\n--------------\nself.weights: {}\nself.value: {}\n--------------\n"
        if (self.input_layer):
            message = "\n--------------\ninput layer: True\nself.value: {}\n--------------\n"
            return message.format(self.value)
        return message.format(self.weights, self.value)
    
        
    def activate(self, inputs = None):
        if (self.input_layer):
            return self.value
        self.setValue(self.activation_function(mult_and_sum(inputs, self.weights)))
        return self.value
    
    def update_weights(self, inp, update):
        # I should update the weight only if the reward is bigger? 
        errors = self.learning_rate * (update - self.activate(inp))
        self.weights = np.add(self.weights, np.multiply(inp, errors))
    
        


In [10]:
def linear_classifiyer(x):
    return x

class Layer(object):
    def __init__(self, neuron_count, input_counts = 1, neuron_weights = None,
                 activation_function = linear_classifiyer):
        random.seed()
        self.neurons = []
        self.input_layer = True if input_counts == 1 else False
        
        for i in range(neuron_count):
            if (neuron_weights != None):
                self.neurons.append(Neuron(input_counts, input_layer = self.input_layer, weights = neuron_weights[i]))
            else:
                self.neurons.append(Neuron(input_counts, input_layer = self.input_layer))
    
    def setValues(self, inputs):
        for index, neuron in enumerate(self.neurons):
            neuron.setValue(inputs[index])
    
    
    def __str__(self):
        message = "[input_layer: {},\nneurons: {}]"
        return message.format(self.input_layer, len(self.neurons))

    def update(self, inputs, reward):
        if (self.input_layer):
            return 
        for neuron in self.neurons:
            neuron.update_weights(inputs, reward)
    
    def output(self, inputs = None):
        layer_output = []
        for neuron in self.neurons:
            layer_output.append(neuron.activate(inputs))
        return layer_output
            
    


In [11]:
# I want to give list of lists and create all layers with all neurons.
# I should be able to use the weights inside the list of lists
# Need an example of weights
# 24 input -> 16 -> 8 -> 4 (output)

# First layer -> no weights
# Second Layer -> 16 neurons each has 24 inputs and 24 weights. 
# third layer -> 8 neurons each has 16 inputs and 16 weights.
# layer #4 -> 4 neurons each has 4 inputs and 4 weights.

#Example

# [
#  [ [w1, w2, w3, ..., ], [], [], ... , []  ] # second layer consist from 16 arrays of weights. each has length == 24 
#  [ [w1, w2, w3, ..., ], [], [], ... , []  ] # third layer and etc...
# ]


In [12]:


class Brain(object):
    def __init__(self, input_counts, weights):
        self.layers = []
        self.input_layer = Layer(neuron_count = input_counts)
        self.layers.append(self.input_layer)
        for i, layerWeight in enumerate(weights):
            layer = layerWeight
            self.layers.append( Layer(neuron_count = len(layer), input_counts = len(layer[-1]),
                                      neuron_weights = layer, activation_function = math.tanh))
        self.output_layer = self.layers[-1]
        
    def __str__(self):
        message = "\n\nlayers: {}\ninput_counts: {},\noutput_counts: {},\nhidden_layers: {}\n\n"
        return message.format(len(self.layers),
                              len(self.layers[0].neurons),
                              len(self.layers[-1].neurons),
                              len(self.layers) - 2)
    
    def generate_action(self, data):
        self.input_layer.setValues(data)
        inputs  = self.input_layer.output()
        for hidden_layer in self.layers[1: -1]:
            inputs = hidden_layer.output(inputs)
        return self.output_layer.output(inputs)
    
    def learn(self, observation, action, reward):
        data = np.concatenate((observation, action), axis=None)
        self.input_layer.setValues(data)
        layer_input = self.input_layer.output()
        for layer in self.layers[1:]:
            layer.update(layer_input, reward)
            layer_input = layer.output(layer_input)

In [14]:
def initializeRandomWeights(schema):
    weights = []
    for layer in schema:
        neurons = []
        for neuron in layer:
            neuront = []
            for k in range(neuron[0]):
                neuront.append(random.uniform(-1, 1))
            neurons.append(neuront)
        weights.append(neurons)
    return weights

def createSchema(layers = 1, neurons = [1]):
    schema = []
    layer_len = neurons[0]
    for n in neurons:
        layer = []
        
        for _ in range(n):
            layer.append([layer_len])
        layer_len = len(layer)
        schema.append(layer)
    return schema

def convertWeightsToNumpy(weights):
    w = []
    for layer in weights:
        for neuron in layer:
            for element in neuron:
                w.append(element)
    return np.array(w)

def convertWeightsToList(schema, w):
    weights = []
    i = 0
    for layer in schema:
        neurons = []
        for neuron in layer:
            neuront = []
            for k in range(neuron[0]):
                neuront.append(w[i])
                i = i + 1
            neurons.append(neuront)
        weights.append(neurons)
    return weights


In [23]:
def f(w):
    observation = env.reset()
    tester = Brain(input_counts = 24, weights = w)
    total = 0
    for i in range(500):
        action = tester.generate_action(observation)
        new_observation, reward, done, info = env.step(action)
        observation = new_observation
        total += reward
        if (done == True or reward == -100 or total > 100):
            break
    return (total, i)


In [20]:
data = pd.read_csv('./weights.csv')
d = np.array(data["weight"])


In [ ]:
npop = 50 # population size
sigma = 0.1 # noise standard deviation
alpha = 0.05 # learning rate
np.random.seed(422142)

schema = createSchema(layers=3, neurons = [24, 6, 4])
w = convertWeightsToNumpy(initializeRandomWeights(schema))
#w = d
i = 0
while (True):
    N = -2 * np.random.random_sample((npop, w.shape[0])) + 1
    R = np.zeros(npop)
    steps = 0
    reward = 0
    for j in range(npop):
        w_try = w + sigma*N[j]
        R[j], s = f(convertWeightsToList(schema, w_try))
        steps += s
        reward += R[j]
    if (i % 10 == 0):
        print('Episode: %d, Steps: %.1f, Reward: %.1f' % (i + 1, steps / npop, reward / npop))
    A = (R - np.mean(R)) / np.std(R)
    w = w + alpha / (npop*sigma) * np.dot(N.T, A)
    if (reward / npop > 100):
        break
    i += 1

print("Episodes: %d" % (i + 1))

Episode: 1, Steps: 94.0, Reward: -116.9
Episode: 11, Steps: 98.8, Reward: -102.8
Episode: 21, Steps: 95.9, Reward: -99.2
Episode: 31, Steps: 409.4, Reward: -50.8
Episode: 41, Steps: 499.0, Reward: -19.5


In [22]:
data = pd.DataFrame(w, columns=['weight'])
data.to_csv('./weights.csv')

In [ ]:
weights = convertWeightsToList(schema, w)
marvin = Brain(input_counts = 24, weights = weights)
total = 0
observation = env.reset()
for step in range(2000):
    env.render()
    
    action = marvin.generate_action(observation)
    
    new_observation, reward, done, info = env.step(action)
    total += reward
    if (done == True):
        break 
env.close()
print(step, total)